In [1]:
import pandas as pd
def get_corpus():
  df = pd.read_csv("https://raw.githubusercontent.com/bshmueli/108-nlp/master/reuters.csv")
  print("Dataset size:",len(df))
  print("Dataset columns:",df.columns)
  corpus=df['content'].to_list()
  title = df['title'].to_list()
  return corpus,title
data,title=get_corpus()
def get_stopword():
  df = pd.read_csv("https://raw.githubusercontent.com/bshmueli/108-nlp/master/stopwords.txt",header=None)
  print(df.columns)
  corpus= df[0].to_list()
  return corpus
stop_word = get_stopword()

Dataset size: 5354
Dataset columns: Index(['title', 'content'], dtype='object')
Int64Index([0], dtype='int64')


In [2]:
import string
punc=list(set(string.punctuation))
punc.append('"')

In [3]:
print(punc)

[':', '>', '#', ',', '&', "'", '}', '<', '~', '"', '^', '!', '+', '\\', ';', '@', ']', '?', '`', '{', '(', '_', '/', '.', ')', '[', '-', '|', '$', '%', '=', '*', '"']


### Clear Punc

In [4]:
import re
def clear_punc(corpus,target_punc):
  data=[]
  for single_docu in corpus:
    single_clear_docu=''
    for element in single_docu:
      for j in range(len(target_punc)):
        if element == str(target_punc[j]):
          element=None
          break
      if element!=None:  
        single_clear_docu+= element
    single_clear=re.sub("”".encode().decode("utf8"), "".encode().decode("utf8"),single_clear_docu.encode().decode("utf-8"))
    data.append(single_clear)
  return data
#clear_data=clear_punc(data[:10],punc)
#print(data[0])
#print(clear_data[0])

### Convert to Lower

In [5]:
def lower_(corpus):
  corp=[]
  for docx in corpus:
    tmpstr=''
    for word in docx:
      if ord(word)>64 and ord(word)<91:
        lower=chr(ord(word)+32)
        tmpstr+=lower
      else:
        tmpstr+=word
    corp.append(tmpstr)
  return corp
#lower_data=lower_(clear_data)
#print(lower_data[0])

### Remove_stop word

In [6]:
def tokenize(document):
  words = document.split(' ')
  return words

In [7]:
stop_word.append('u')
def clear_stop_word(corpus,target_stop_word):
  corp=[]
  for docx in corpus:
    tokens=tokenize(docx)
    tmp=[]
    for token in tokens:
      flag=0
      for stop_word in target_stop_word:
        if str(token) == str(stop_word) or token=='':
          flag=1
          break
      if flag!=1:
        tmp.append(token)
    corp.append(tmp)
  return corp
#no_stop_corpus=clear_stop_word(lower_data,stop_word)


In [8]:
#print(no_stop_corpus[0])

### TF-IDF

In [9]:
from collections import Counter
def get_vocab(corpus):
  vocabulary = Counter()
  for document in corpus:
    vocabulary.update(document)
  return vocabulary
#vocab=get_vocab(no_stop_corpus)

In [10]:
def tf(corpus):
  tf_dict=[]
  for docx in corpus:
    docx_word_dict=Counter(docx)
    tfdict=dict.fromkeys(docx_word_dict,0)
    word_count=len(docx)
    for word,count in docx_word_dict.items():
      tfdict[word]=count / float(word_count)
    tf_dict.append(tfdict)
  return tf_dict
#tf_score=tf(no_stop_corpus)
#print(tf_score[0])


In [11]:
#print(vocab)

In [12]:
import math
def idf(tf_scores,total_vocab):
  idf_dict=[]
  N=len(tf_scores)
  idfdict = dict.fromkeys(total_vocab.keys(), 0)
  for tf_score in tf_scores:
    for word, val in tf_score.items():
      #print(word,val)
      if val > 0:
        idfdict[word] += 1
        #print(idfdict)
  for word, val in idfdict.items():
    idfdict[word] = math.log(N / float(val))
  #print(idfdict)
  return idfdict
#idf_score=idf(tf_score,vocab)


In [13]:
#print(idf_score['hey'])

In [14]:
def tf_idf(tf,idf):
  tf_idf_dict=[]
  for docx_tf in tf:
    tmp_dict={}
    for word ,val in docx_tf.items():
      tmp_dict[word]=val *idf[word]
    tf_idf_dict.append(tmp_dict)
  return tf_idf_dict
#tf_idf_score=tf_idf(tf_score,idf_score)

In [15]:
#print(tf_idf_score[0])

### Get top 1000 per document tf-idf scores

In [16]:
def get_top(tf_corpus_dict,top=1000): 
  tops=[]
  for tf_dict in tf_corpus_dict:
    sorted_ = sorted(tf_dict.items(),reverse=True, key=lambda key: key[1])
    keyList = [] 
    valueList = [] 
    i=0
    for key,value in sorted_:
      i+=1
      keyList.append(key) 
      if i<top:
        valueList.append(value) 
      else:
        valueList.append(float(0))
    top_tf_idf = dict(zip(keyList, valueList))
    tops.append(top_tf_idf)
  return tops
#tf_idf_top_score=get_top(tf_idf_score)

In [17]:
def doc2vec(i):
  #words = tokenize(doc)
  vec=[]
  #print(i)  
  for token ,_ in vocab.items():
    #print(i,token)
    if token in no_stop_corpus[i]:
      vec.append(tf_idf_top_score[i][token])
    else:
      vec.append(float(0))
  return vec
 # return [score if token in no_stop_corpus[i] else 0 for token,score in tf_idf_top_score[i]]

In [18]:
def cosine_similarity(vec_a,vec_b):
  assert len(vec_a) == len(vec_b)
  if (sum(vec_a) == 0 or sum(vec_b)==0):
    return 0
  a_b=sum(i[0] * i[1] for i in zip(vec_a,vec_b))
  a_2=sum(i*i for i in vec_a)
  b_2=sum(i*i for i in vec_b)
  return a_b/(math.sqrt(a_2)*math.sqrt(b_2))

In [19]:
def doc_similarity(doc_a,doc_b):
  return cosine_similarity(doc2vec(doc_a),doc2vec(doc_b))

In [25]:
def k_similar(seed_id, k=5):
  seed_doc = title[seed_id]
  print('> "{}"'.format(seed_doc))
  similarities = [doc_similarity(seed_id,doc) for doc in range(len(data))]
  top_indices = sorted(range(len(similarities)),key = lambda i : similarities[i])[-k:]
  nearest = [[title[id],similarities[id]] for id in top_indices]
  for story in reversed(nearest):
    print('*"{}"({})'.format(story[0],story[1]))

In [26]:
clear_data=clear_punc(data,punc)
lower_data=lower_(clear_data)
no_stop_corpus=clear_stop_word(lower_data,stop_word)
vocab=dict(get_vocab(no_stop_corpus))
tf_score=tf(no_stop_corpus)
idf_score=idf(tf_score,vocab)
tf_idf_score=tf_idf(tf_score,idf_score)
tf_idf_top_score=get_top(tf_idf_score)
k_similar(719)


> "Oil slips, inventories curb recovery from four-month lows"
*"Oil slips, inventories curb recovery from four-month lows"(1.0000000000000002)
*"Oil drops to lowest since November as U.S. inventories swell"(0.6067763300105031)
*"Oil jumps over 10 percent as OPEC finalizes output cut deal"(0.49123403216885536)
*"Oil falls on output cut skepticism, OPEC and Russia output rise"(0.45928501328511667)
*"Oil falls $2 a barrel on OPEC cut uncertainty ahead of meeting"(0.45830258845520333)


In [22]:
print(doc_similarity(719,719))
#print(doc2vec(0))
> "Oil slips, inventories curb recovery from four-month lows"
*"Oil slips, inventories curb recovery from four-month lows"(1.0000000000000002)
*"Oil drops to lowest since November as U.S. inventories swell"(0.6067763300105031)
*"Oil jumps over 10 percent as OPEC finalizes output cut deal"(0.49123403216885536)
*"Oil falls on output cut skepticism, OPEC and Russia output rise"(0.45928501328511667)
*"Oil falls $2 a barrel on OPEC cut uncertainty ahead of meeting"(0.45830258845520333)

1.0000000000000002


In [23]:
#print(no_stop_corpus[0])
